In [0]:
import numpy as np
import pandas as pd

In [0]:
INPUT_DIR = 'test_drive/titanic'

train = pd.read_csv(f'{INPUT_DIR}/train.csv')
test = pd.read_csv(f'{INPUT_DIR}/test.csv')

In [0]:
train['boy'] = train['Name'].str.contains('Master', regex=False)
train['boy'] = [1 if x == True else 0 for x in train['boy']]

test['boy'] = test['Name'].str.contains('Master', regex=False)
test['boy'] = [1 if x == True else 0 for x in test['boy']]

In [0]:
train['surname'] = train['Name'].str.split(',', expand=True)[0]
test['surname'] = test['Name'].str.split(',', expand=True)[0]

In [0]:
# remove unwanted columns
feature_to_keep = ['Pclass', 'Sex', 'SibSp', 'Parch','Fare','Embarked']

X_train = train[feature_to_keep]
Y_train = train[['Survived']]
X_test = test[feature_to_keep]

X_train = pd.get_dummies(X_train, columns=["Pclass",'Sex', "Embarked"], prefix=["Pclass", 'Sex', "Embarked"])
X_test = pd.get_dummies(X_test, columns=["Pclass",'Sex', "Embarked"], prefix=["Pclass", 'Sex', "Embarked"])

In [0]:
X_test = X_test.fillna(0)

In [373]:
# scaling some columns using minMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[['SibSp','Fare','Parch']] = scaler.fit_transform(X_train[['SibSp','Fare','Parch']])
X_test[['SibSp','Fare','Parch']] = scaler.fit_transform(X_test[['SibSp','Fare','Parch']])
X_train.head(10)

,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.125,0.000000,0.014151,0,0,1,0,1,0,0,1
1,0.125,0.000000,0.139136,1,0,0,1,0,1,0,0
2,0.000,0.000000,0.015469,0,0,1,1,0,0,0,1
3,0.125,0.000000,0.103644,1,0,0,1,0,0,0,1
4,0.000,0.000000,0.015713,0,0,1,0,1,0,0,1
5,0.000,0.000000,0.016510,0,0,1,0,1,0,1,0
6,0.000,0.000000,0.101229,1,0,0,0,1,0,0,1
7,0.375,0.166667,0.041136,0,0,1,0,1,0,0,1
8,0.000,0.333333,0.021731,0,0,1,1,0,0,0,1
9,0.125,0.000000,0.058694,0,1,0,1,0,1,0,0


In [374]:
####### Evaluate model with dev set ######## 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
clf = GradientBoostingClassifier(learning_rate=0.04, n_estimators=40, max_depth=5)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [375]:
m_test = test.shape[0]
test['Survived'] = 0

for i in range(m_test):
    surname = test.loc[i,'surname']
    if (test.loc[i,'Sex'] == 'male'):
        num_female = train[(train['surname'] == surname) & (train['Sex'] == 'female')].shape[0]
        num_boy = train[(train['surname'] == surname) & (train['boy'] == 1)].shape[0]
        
        num_female_lived = train[(train['surname'] == surname) & (train['Sex'] == 'female') & (train['Survived'] == 1)].shape[0]
        num_boy_lived = train[(train['surname'] == surname) & (train['boy'] == 1) & (train['Survived'] == 1)].shape[0]
        
#         print('num_families ', num_families)
#         print('num_female_died ', num_female_died)
#         print('num_boy_died', num_boy_died)
        # default: man dies
        # only when everyone in his family lives, a boy lives
        if (num_female_lived==num_female and num_boy_lived==num_boy and test.loc[i,'boy'] == 1):
            test.loc[i,'Survived'] = 1
    else:
        num_female = train[(train['surname'] == surname) & (train['Sex'] == 'female')].shape[0]
        num_boy = train[(train['surname'] == surname) & (train['boy'] == 1)].shape[0]
        
        num_female_died = train[(train['surname'] == surname) & (train['Sex'] == 'female') & (train['Survived'] == 0)].shape[0]
        num_boy_died = train[(train['surname'] == surname) & (train['boy'] == 1) & (train['Survived'] == 0)].shape[0]
        
        # default: woman lives
        test.loc[i,'Survived'] = 1 
        
        # only when all boys and female in the family died, and she does have a family, she dies
        if (num_female_died==num_female and num_boy_died == num_boy and (num_female>0 or num_boy>0)):
            test.loc[i,'Survived'] = 0
       
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,boy,surname,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,Kelly,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,Wilkes,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,Myles,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,Wirz,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,Hirvonen,1


In [0]:
output = test[['PassengerId', 'Survived']]

In [377]:
output.to_csv('naive-titanic.csv', index=False)
import os
print( os.getcwd() )

print( os.listdir() )

from google.colab import files
files.download( "naive-titanic.csv" ) 

/content
['datalab', 'naive-titanic.csv', 'new target.csv', 'test_drive', '.ipython', 'predict.csv', '.gdfuse', '.keras', '.config', '.local', '.cache', '.forever']
